In [6]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import renom as rm
from renom.cuda.cuda import set_cuda_active
set_cuda_active(False)

In [7]:
filename = "./isolet1+2+3+4.data"
labels = []
X = []
y = []

def make_label_idx(filename):
    labels = []
    for line in open(filename, "r"):
        line = line.rstrip()
        label = line.split(",")[-1]
        labels.append(label)
    labels = list(set(labels))
    return list(set(labels))

labels = make_label_idx(filename)
labels = sorted(labels, key=lambda d:int(d.replace(".","").replace(" ","")))

In [8]:
for line in open(filename,"r"):
    line = line.rstrip()
    label = labels.index(line.split(",")[-1])
    features = list(map(float,line.split(",")[:-1]))
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("X_train:{}, y_train:{}, X_test:{}, y_test:{}".format(X_train.shape, y_train.shape,
                                                            X_test.shape, y_test.shape))
lb = LabelBinarizer().fit(y)
labels_train = lb.transform(y_train)
labels_test = lb.transform(y_test)
print("labels_train:{}, labels_test:{}".format(labels_train.shape, labels_test.shape))

X_train:(4990, 617), y_train:(4990,), X_test:(1248, 617), y_test:(1248,)
labels_train:(4990, 26), labels_test:(1248, 26)


In [9]:
output_size = len(labels)
sequential = rm.Sequential([
    rm.Dense(100),
    rm.Relu(),
    rm.Dense(50),
    rm.Relu(),
    rm.Dense(output_size)
])

In [10]:
epoch = 20
batch_size = 128
N = len(X_train)
optimizer = rm.Adagrad(lr=0.01)
for i in range(epoch):
    perm = np.random.permutation(N)
    loss = 0
    for j in range(0, N//batch_size):
        train_batch = X_train[perm[j*batch_size : (j+1)*batch_size]]
        response_batch = labels_train[perm[j*batch_size : (j+1)*batch_size]]
        with sequential.train():
            l = rm.softmax_cross_entropy(sequential(train_batch), response_batch)
        grad = l.grad()
        grad.update(optimizer)
        loss += l.as_ndarray()
    train_loss = loss / (N//batch_size)
    test_loss = rm.softmax_cross_entropy(sequential(X_test), labels_test).as_ndarray()
    print("epoch:{:03d}, train_loss:{:.4f}, test_loss:{:.4f}".format(i, float(train_loss), float(test_loss)))


epoch:000, train_loss:1.4529, test_loss:0.6893
epoch:001, train_loss:0.5232, test_loss:0.4279
epoch:002, train_loss:0.3581, test_loss:0.3612
epoch:003, train_loss:0.2839, test_loss:0.3005
epoch:004, train_loss:0.2332, test_loss:0.2766
epoch:005, train_loss:0.2021, test_loss:0.2508
epoch:006, train_loss:0.1784, test_loss:0.2332
epoch:007, train_loss:0.1613, test_loss:0.2256
epoch:008, train_loss:0.1433, test_loss:0.2157
epoch:009, train_loss:0.1367, test_loss:0.2093
epoch:010, train_loss:0.1215, test_loss:0.2021
epoch:011, train_loss:0.1126, test_loss:0.1979
epoch:012, train_loss:0.1049, test_loss:0.1994
epoch:013, train_loss:0.0990, test_loss:0.1996
epoch:014, train_loss:0.0936, test_loss:0.1869
epoch:015, train_loss:0.0888, test_loss:0.1844
epoch:016, train_loss:0.0834, test_loss:0.1863
epoch:017, train_loss:0.0787, test_loss:0.1808
epoch:018, train_loss:0.0747, test_loss:0.1763
epoch:019, train_loss:0.0707, test_loss:0.1774
